## Neural networks for Face AE

Here is an example of input data:

In [ ]:
import scipy as sp
import scipy.misc
import matplotlib.pyplot as plt
import numpy as np
from imageio import imread
%matplotlib inline
from __future__ import print_function
import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import time
from glob import glob
import numpy as npCan't get attribute
from PIL import Image
from os import listdir
from os.path import isfile, join
import face_recognition

In [ ]:
# Human HT29 colon-cancer cells
plt.figure(figsize=(10,8))
plt.subplot(1,2,1)
im = imread('./face_image/000001.jpg')
plt.imshow(im)
plt.subplot(1,2,2)
mask = imread('./face_image/000005.jpg')
plt.imshow(mask)

In [ ]:
img = Image.open('./face_image/000001.jpg')

In [ ]:
img.size

In [ ]:
img.resize((128,128))

In [ ]:
# Archiecture from here https://github.com/bhpfelix/Variational-Autoencoder-PyTorch
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class VAE(nn.Module):
    def __init__(self, nc, ngf, ndf, latent_variable_size):
        super(VAE, self).__init__()

        self.nc = nc
        self.ngf = ngf
        self.ndf = ndf
        self.latent_variable_size = latent_variable_size

        # encoder
        self.e1 = nn.Conv2d(nc, ndf, 4, 2, 1)
        self.bn1 = nn.BatchNorm2d(ndf)

        self.e2 = nn.Conv2d(ndf, ndf*2, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(ndf*2)

        self.e3 = nn.Conv2d(ndf*2, ndf*4, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ndf*4)

        self.e4 = nn.Conv2d(ndf*4, ndf*8, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(ndf*8)

        self.e5 = nn.Conv2d(ndf*8, ndf*8, 4, 2, 1)
        self.bn5 = nn.BatchNorm2d(ndf*8)

        self.fc1 = nn.Linear(ndf*8*4*4, latent_variable_size)
        self.fc2 = nn.Linear(ndf*8*4*4, latent_variable_size)

        # decoder
        self.d1 = nn.Linear(latent_variable_size, ngf*8*2*4*4)

        self.up1 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd1 = nn.ReplicationPad2d(1)
        self.d2 = nn.Conv2d(ngf*8*2, ngf*8, 3, 1)
        self.bn6 = nn.BatchNorm2d(ngf*8, 1.e-3)

        self.up2 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd2 = nn.ReplicationPad2d(1)
        self.d3 = nn.Conv2d(ngf*8, ngf*4, 3, 1)
        self.bn7 = nn.BatchNorm2d(ngf*4, 1.e-3)

        self.up3 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd3 = nn.ReplicationPad2d(1)
        self.d4 = nn.Conv2d(ngf*4, ngf*2, 3, 1)
        self.bn8 = nn.BatchNorm2d(ngf*2, 1.e-3)

        self.up4 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd4 = nn.ReplicationPad2d(1)
        self.d5 = nn.Conv2d(ngf*2, ngf, 3, 1)
        self.bn9 = nn.BatchNorm2d(ngf, 1.e-3)

        self.up5 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd5 = nn.ReplicationPad2d(1)
        self.d6 = nn.Conv2d(ngf, nc, 3, 1)

        self.leakyrelu = nn.LeakyReLU(0.2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.leakyrelu(self.bn1(self.e1(x)))
        h2 = self.leakyrelu(self.bn2(self.e2(h1)))
        h3 = self.leakyrelu(self.bn3(self.e3(h2)))
        h4 = self.leakyrelu(self.bn4(self.e4(h3)))
        h5 = self.leakyrelu(self.bn5(self.e5(h4)))
        h5 = h5.view(-1, self.ndf*8*4*4)

        return self.fc1(h5), self.fc2(h5)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if args.cuda:
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h1 = self.relu(self.d1(z))
        h1 = h1.view(-1, self.ngf*8*2, 4, 4)
        h2 = self.leakyrelu(self.bn6(self.d2(self.pd1(self.up1(h1)))))
        h3 = self.leakyrelu(self.bn7(self.d3(self.pd2(self.up2(h2)))))
        h4 = self.leakyrelu(self.bn8(self.d4(self.pd3(self.up3(h3)))))
        h5 = self.leakyrelu(self.bn9(self.d5(self.pd4(self.up4(h4)))))

        return self.sigmoid(self.d6(self.pd5(self.up5(h5))))

    def get_latent_var(self, x):
        mu, logvar = self.encode(x.view(-1, self.nc, self.ndf, self.ngf))
        z = self.reparametrize(mu, logvar)
        return z

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.nc, self.ndf, self.ngf))
        z = self.reparametrize(mu, logvar)
        res = self.decode(z)
        return res, mu, logvar

In [ ]:
model = VAE(nc=3, ngf=128, ndf=128, latent_variable_size=128).cuda()

In [ ]:
# model=torch.load('model')
totensor = transforms.ToTensor()

In [ ]:
from torch import optim as opt
reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False
def loss_function(recon_x, x, mu, logvar):
    BCE = reconstruction_function(recon_x, x)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return BCE + KLD

optimizer = opt.Adam(model.parameters(), lr=1e-4)

In [ ]:
class Object(object):
    pass

args = Object()
args.cuda = True
args.log_interval=20

In [ ]:
from torch import np # Torch wrapper for Numpy

import os
from PIL import Image

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
a=0
from sklearn.preprocessing import MultiLabelBinarizer
class MakeData(Dataset):
    def __init__(self,  img_path,img_y_path, transform=None,tr=None):
        
        tmp_df_x = os.listdir(img_path)
        tmp_df_x.sort()
        
        tmp_df_y = os.listdir(img_y_path)
        tmp_df_y.sort()
        
        self.img_path_x = img_path
        self.img_path_y = img_y_path
        self.transform = transform
        self.tr =tr
        self.X_train = tmp_df_x
        self.y_train = tmp_df_y
        self.trans =transforms.Compose([
            transforms.ToTensor(),
        ])
       
    def __getitem__(self, index):
        img = Image.open(self.img_path_x + self.X_train[index])
        img = img.convert('RGB')
        img = img.resize((128,128))
        img = self.trans(img)
        return img

    def __len__(self):
        return len(self.X_train) 

In [ ]:
pathtrain = "face_image/"
train_transform = transforms.Compose([
            transforms.ToTensor(),
        ])
dset_train = MakeData(pathtrain,pathtrain,train_transform,tr=1)

In [ ]:
train_loader = DataLoader(dset_train,
                          batch_size=64,
                          shuffle=True,
                          num_workers=1, # 1 for CUDA
                          pin_memory=True # CUDA only
                         )

In [ ]:
import time
num_epochs = 150 # total amount of full passes over training data
batch_size = 32
train_loss=[]
for epoch in range(num_epochs):

    start_time = time.time()
    model.train(True) 
    i=0
    for X_batch in train_loader:
        # train on batch
        if args.cuda:
            X_batch = Variable(X_batch.cuda())
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(X_batch)
        loss = loss_function(recon_batch, X_batch, mu, logvar)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.cpu().data.numpy()[0])
        i+=1
        if(i%100==0):
            print(np.mean(train_loss[-1]))
        del loss
    torch.save(model,'model')
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration): \t{:.6f}".format(
        np.mean(train_loss[- 202000// batch_size :])))
